# LAB - 3 TDL
---

    Ayush Singh
    PES2UG20CS081
    SECTION -6B

---

In [4]:
# baseline cnn model for fashion mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import Adam

2023-02-08 09:52:08.206869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [6]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [7]:
# define cnn model
def define_model():
	model = Sequential([
		Conv2D(16, (4,4), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),
		Conv2D(8, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),
		MaxPooling2D((2, 2)),
		Flatten(),
		Dense(100, activation='relu', kernel_initializer='he_uniform'),
		Dense(10, activation='softmax')
	])

	# compile model
	opt = Adam(learning_rate=0.01)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	print(model.summary())
	return model


In [8]:
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=1)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# append scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

In [9]:
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(211)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(212)
		pyplot.title('ClassificConv2D(32, (4,4), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),ation Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

In [10]:
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

In [11]:
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

In [12]:
run_test_harness()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 100)               540900    
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 542,230
Trainable params: 542,230
Non-trainable params: 0
__________________________________________________

2023-02-08 09:52:11.496669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 09:52:11.497796: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


None
Epoch 1/10
1500/1500 [==============================] - 76s 50ms/step - loss: 0.4785 - accuracy: 0.8505 - val_loss: 0.3258 - val_accuracy: 0.8799
Epoch 2/10
1500/1500 [==============================] - 83s 56ms/step - loss: 0.2942 - accuracy: 0.8917 - val_loss: 0.3028 - val_accuracy: 0.8875
Epoch 3/10
1500/1500 [==============================] - 70s 47ms/step - loss: 0.2648 - accuracy: 0.9022 - val_loss: 0.3177 - val_accuracy: 0.8894
Epoch 4/10
1500/1500 [==============================] - 82s 55ms/step - loss: 0.2430 - accuracy: 0.9099 - val_loss: 0.3276 - val_accuracy: 0.8911
Epoch 5/10
1500/1500 [==============================] - 86s 57ms/step - loss: 0.2214 - accuracy: 0.9173 - val_loss: 0.3563 - val_accuracy: 0.8897
Epoch 6/10
1500/1500 [==============================] - 81s 54ms/step - loss: 0.2101 - accuracy: 0.9219 - val_loss: 0.3768 - val_accuracy: 0.8833
Epoch 7/10
1500/1500 [==============================] - 78s 52ms/step - loss: 0.1913 - accuracy: 0.9287 - val_loss: 0.3